In [3]:
!mkdir -p data/mnist

```shell

wget -c -P data/mnist http://yann.lecun.com/exdb/mnist/{train-images-idx3-ubyte.gz,train-labels-idx1-ubyte.gz,t10k-images-idx3-ubyte.gz,t10k-labels-idx1-ubyte.gz}

Output:

deeplearn@Ubuntu-1604-xenial-64-minimal:~/deepfun/tensorflow/CapsNets$ wget -c -P data/mnist http://yann.lecun.com/exdb/mnist/{train-images-idx3-ubyte.gz,train-labels-idx1-ubyte.gz,t10k-images-idx3-ubyte.gz,t10k-labels-idx1-ubyte.gz}
--2017-11-25 00:13:37--  http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘data/mnist/train-images-idx3-ubyte.gz’

train-images-idx3-ubyte.gz             100%[============================================================================>]   9.45M  8.29MB/s    in 1.1s    

2017-11-25 00:13:39 (8.29 MB/s) - ‘data/mnist/train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2017-11-25 00:13:39--  http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Reusing existing connection to yann.lecun.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 28881 (28K) [application/x-gzip]
Saving to: ‘data/mnist/train-labels-idx1-ubyte.gz’

train-labels-idx1-ubyte.gz             100%[============================================================================>]  28.20K  --.-KB/s    in 0s      

2017-11-25 00:13:39 (353 MB/s) - ‘data/mnist/train-labels-idx1-ubyte.gz’ saved [28881/28881]

--2017-11-25 00:13:39--  http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
Reusing existing connection to yann.lecun.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 1648877 (1.6M) [application/x-gzip]
Saving to: ‘data/mnist/t10k-images-idx3-ubyte.gz’

t10k-images-idx3-ubyte.gz              100%[============================================================================>]   1.57M  --.-KB/s    in 0.01s   

2017-11-25 00:13:39 (112 MB/s) - ‘data/mnist/t10k-images-idx3-ubyte.gz’ saved [1648877/1648877]

--2017-11-25 00:13:39--  http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Reusing existing connection to yann.lecun.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 4542 (4.4K) [application/x-gzip]
Saving to: ‘data/mnist/t10k-labels-idx1-ubyte.gz’

t10k-labels-idx1-ubyte.gz              100%[============================================================================>]   4.44K  --.-KB/s    in 0s      

2017-11-25 00:13:39 (337 MB/s) - ‘data/mnist/t10k-labels-idx1-ubyte.gz’ saved [4542/4542]

FINISHED --2017-11-25 00:13:39--
Total wall clock time: 1.6s
Downloaded: 4 files, 11M in 1.2s (9.57 MB/s)
```
and then unzip it

```shell

gunzip data/mnist/*.gz

```



In [12]:
import os
import tensorflow as tf
from tqdm import tqdm
from config import cfg
from utils import load_mnist
from capsNet import CapsNet

In [13]:
capsNet = CapsNet(is_training=cfg.is_training)
tf.logging.info('Graph loaded')

INFO:tensorflow:Seting up the main structure
INFO:tensorflow:Graph loaded


In [14]:
sv = tf.train.Supervisor(graph=capsNet.graph,
                             logdir=cfg.logdir,
                             save_model_secs=0)

path = cfg.results + '/accuracy.csv'
if not os.path.exists(cfg.results):
    os.mkdir(cfg.results)
elif os.path.exists(path):
    os.remove(path)

In [15]:
fd_results = open(path, 'w')
fd_results.write('step,test_acc\n')
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
with sv.managed_session(config=config) as sess:
        num_batch = int(60000 / cfg.batch_size)
        num_test_batch = 10000 // cfg.batch_size
        teX, teY = load_mnist(cfg.dataset, False)
        for epoch in range(cfg.epoch):
            if sv.should_stop():
                break
            for step in tqdm(range(num_batch), total=num_batch, ncols=70, leave=False, unit='b'):
                global_step = sess.run(capsNet.global_step)
                sess.run(capsNet.train_op)

                if step % cfg.train_sum_freq == 0:
                    _, summary_str = sess.run([capsNet.train_op, capsNet.train_summary])
                    sv.summary_writer.add_summary(summary_str, global_step)

                if (global_step + 1) % cfg.test_sum_freq == 0:
                    test_acc = 0
                    for i in range(num_test_batch):
                        start = i * cfg.batch_size
                        end = start + cfg.batch_size
                        test_acc += sess.run(capsNet.batch_accuracy, {capsNet.X: teX[start:end], capsNet.labels: teY[start:end]})
                    test_acc = test_acc / (cfg.batch_size * num_test_batch)
                    fd_results.write(str(global_step + 1) + ',' + str(test_acc) + '\n')
                    fd_results.flush()

            if epoch % cfg.save_freq == 0:
                sv.saver.save(sess, cfg.logdir + '/model_epoch_%04d_step_%02d' % (epoch, global_step))

fd_results.close()
tf.logging.info('Training done')

INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.



  0%|                                          | 0/234 [00:00<?, ?b/s]

INFO:tensorflow:Recording summary at step 0.



  2%|▋                                 | 5/234 [00:06<05:10,  1.36s/b]
Exception in thread Thread-27:                                        
Traceback (most recent call last):
  File "/home/deeplearn/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/deeplearn/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/deeplearn/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 70%|██████████████████████▎         | 163/234 [02:00<00:52,  1.36b/s]

INFO:tensorflow:Recording summary at step 167.


...

 60%|███████████████████▎            | 141/234 [01:41<01:06,  1.39b/s]

INFO:tensorflow:Recording summary at step 106500.


 23%|███████▌                         | 54/234 [00:39<02:11,  1.37b/s]

INFO:tensorflow:Recording summary at step 106650.


 78%|█████████████████████████       | 183/234 [02:11<00:36,  1.39b/s]